# REST API Calls with AML on US Gov

This notebook arrived out of troubleshooting Promptflow on AML Gov:

1. The Promptflow sdk does not currently support Azure US Gov so the calls containing `flows/api` demonstrate a workaround with the REST API.
2. The US Gov AML Promptflow UI does not support AOAI connections outside of the gov subcription so the last call shows how one can be created with the REST API as AOAI in US Gov was not supported at the time of writing.

A helpful file for the AML Promptflow REST API is this [swagger.json](https://github.com/microsoft/promptflow/blob/ee636fc644e6a5a41f9aa473d7befe1f099beea2/src/promptflow/promptflow/azure/_restclient/swagger.json).

In [ ]:
# !pip install azure-ai-ml

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from dotenv import load_dotenv, find_dotenv


# assert's that parent directory of this notebook is in python path
# need this to locally import ml/src folder
project_directory = "/workspaces/llmops-promptflow-template"
if project_directory not in sys.path:
    sys.path.append(project_directory)


try:
    # API_KEY = os.getenv("AOAI_KEY")
    # RESOURCE_ENDPOINT = os.getenv("AOAI_ENDPOINT")
    _ = load_dotenv(find_dotenv())  # read local .env file

    SUBSCRIPTION_ID = os.getenv("AML_SUBSCRIPTION_ID", "").strip()
    RESOURCE_GROUP = os.getenv("AML_RESOURCE_GROUP", "").strip()
    AML_WORKSPACE_NAME = os.getenv("AML_WORKSPACE_NAME", "").strip()

except:
    raise ValueError("Dotfile containing environment variables not found.")

In [ ]:
%%bash

if [ $(az account show --query id -o tsv) = $AML_SUBSCRIPTION_ID ]; then
    echo "Good to go! You're already logged in to Azure and the subscription has previously been set."
else
    echo "Please login to Azure."
    az cloud set -n "AzureUSGovernment"
    az login --use-device-code --output none
    az account set -s $AML_SUBSCRIPTION_ID
fi

In [27]:
from azure.identity import (
    DefaultAzureCredential,
    AzureAuthorityHosts,
)

credential = DefaultAzureCredential(authority=AzureAuthorityHosts.AZURE_GOVERNMENT)

In [ ]:
import logging

logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.WARNING)
requests_log.propagate = True

## Create a runtime in AML

Currently we are unable to get Automatic Runtimes working via PF, so we need to use a custom runtime. Using the Rest API allows you to create a runtime using the Service Principal

In [31]:
# Set variables for app and runtime
subscription_id = SUBSCRIPTION_ID
resource_group = RESOURCE_GROUP
workspace_name = AML_WORKSPACE_NAME
compute_name = "llmops-compute"
runtime_name = "llmops_test_api_runtime"
custom_app_name = "llmops-app-2"
pf_docker_image = "mcr.microsoft.com/azureml/promptflow/promptflow-runtime:latest"
# Client ID for Managed Identity of the compute instance
mi_client_id = "71574d03-0491-453d-bcbf-22a91d28a82a"
# important variable for runtime configs
prt_config_override = (
    f"storage.storage_account={workspace_name},"
    f"deployment.subscription_id={subscription_id},"
    f"deployment.resource_group={resource_group},"
    f"deployment.workspace_name={workspace_name},"
    f"deployment.endpoint_name={compute_name},"
    f"deployment.deployment_name={runtime_name},"
    f"deployment.mt_service_endpoint=https://usgovvirginia.api.ml.azure.us,"
    f"deployment.runtime_name={runtime_name}"
)

In [32]:
import requests
import json

# Get the compute session
# , in the response there is a field for customServices. responses - content - properties - properties - customServices
# Take that list, append the new one. Then post to the rest endpoint 
# https://management.azure.com/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/providers/Microsoft.MachineLearningServices/workspaces/{workspaceName}/computes/{computeName}/customServices

url = (
    f"https://management.usgovcloudapi.net/subscriptions/{subscription_id}/"
    f"resourcegroups/{resource_group}/providers/Microsoft.MachineLearningServices/"
    f"workspaces/{workspace_name}/computes/{compute_name}"
    f"?api-version=2023-04-01-preview"
)
token = credential.get_token("https://management.usgovcloudapi.net/.default").token
header = {
    "Authorization": f"Bearer {token}",
    "content-type": "application/json",
}

with requests.Session() as session:
    response = session.get(url, headers=header)
    # Raise an exception if the response contains an HTTP error status code
    response.raise_for_status()

print(response.json())

# Json for custom services
service_json = {
                                "type": "docker",
                                "name": custom_app_name,
                                "image": {
                                    "type": "docker",
                                    "reference": pf_docker_image
                                },
                                "environmentVariables": {
                                    "AZURE_CLIENT_ID": {
                                        "type": "local",
                                        "value": mi_client_id
                                    },
                                    "AZURE_RESOURCE_MANAGER": {
                                        "type": "local",
                                        "value": "https://management.usgovcloudapi.net/"
                                    },
                                    "AZUREML_CURRENT_CLOUD": {
                                        "type": "local",
                                        "value": "AzureUSGovernment"
                                    },
                                    "PRT_CONFIG_OVERRIDE": {
                                        "type": "local",
                                        "value": prt_config_override
                                    }
                                },
                                "docker": {
                                    "privileged": True
                                },
                                "endpoints": [
                                    {
                                        "protocol": "http",
                                        "name": "connect",
                                        "target": 8080,
                                        "published": 8090,
                                        "hostIp": None
                                    }
                                ],
                                "volumes": [
                                    {
                                        "type": "bind",
                                        "readOnly": None,
                                        "source": "/var/run/docker.sock",
                                        "target": "/var/run/docker.sock",
                                        "consistency": None,
                                        "bind": None,
                                        "volume": None,
                                        "tmpfs": None
                                    }
                                ],
                                "kernel": None
                            }

custom_services = response.json()["properties"]["properties"]["customServices"]
custom_services.append(service_json)

url = (
    f"https://management.usgovcloudapi.net/subscriptions/{subscription_id}/"
    f"resourcegroups/{resource_group}/providers/Microsoft.MachineLearningServices/"
    f"workspaces/{workspace_name}/computes/{compute_name}/customServices"
    f"?api-version=2023-04-01-preview"
)

data = json.dumps(custom_services)
print(url)
print(data)
with requests.Session() as session:
    response = session.post(url, data=data, headers=header)
    # Raise an exception if the response contains an HTTP error status code
    response.raise_for_status()

{'id': '/subscriptions/50ff9458-6372-4522-8227-327043deaef5/resourceGroups/ngc-eng-reqs-mve/providers/Microsoft.MachineLearningServices/workspaces/ngcengreqsmve/computes/llmops-compute', 'name': 'llmops-compute', 'type': 'Microsoft.MachineLearningServices/workspaces/computes', 'location': 'usgovvirginia', 'tags': {}, 'identity': {'type': 'UserAssigned', 'userAssignedIdentities': {'/subscriptions/50ff9458-6372-4522-8227-327043deaef5/resourcegroups/ngc-eng-reqs-mve/providers/Microsoft.ManagedIdentity/userAssignedIdentities/llmops-mi': {'principalId': '12130ef4-89bc-4431-8534-03f1108e2bbd', 'clientId': '71574d03-0491-453d-bcbf-22a91d28a82a'}}}, 'properties': {'createdOn': '2023-12-13T14:46:19.7750191+00:00', 'modifiedOn': '2023-12-13T15:20:19.9685473+00:00', 'disableLocalAuth': False, 'description': None, 'resourceId': None, 'computeType': 'ComputeInstance', 'computeLocation': 'usgovvirginia', 'provisioningState': 'Succeeded', 'provisioningErrors': None, 'provisioningWarnings': {}, 'isAtt

In [34]:
import requests
import json


url = (
    f"https://usgovvirginia.api.ml.azure.us/flow/api/subscriptions/{subscription_id}/"
    f"resourcegroups/{resource_group}/providers/Microsoft.MachineLearningServices/"
    f"workspaces/{workspace_name}/FlowRuntimes/{runtime_name}?asyncCall=true"
)
token = credential.get_token("https://management.usgovcloudapi.net/.default").token
header = {
    "Authorization": f"Bearer {token}",
    "content-type": "application/json",
}

data = json.dumps(
    {
  "runtimeType": "ComputeInstance",
  "identity": {
    "type": "UserAssigned",
    "principalId": "12130ef4-89bc-4431-8534-03f1108e2bbd",
    "tenantId": "71574d03-0491-453d-bcbf-22a91d28a82a"
  },
  "instanceType": "string",
  "fromExistingEndpoint": False,
  "fromExistingDeployment": False,
  "endpointName": runtime_name,
  "deploymentName": runtime_name,
  "computeInstanceName": compute_name,
  "fromExistingCustomApp": True,
  "customAppName": custom_app_name, #TODO: Make custom app
  "runtimeDescription": "This is a custom runtime made for LLMOPS",
  "instanceCount": 1
}
)

with requests.Session() as session:
    response = session.post(url, data=data, headers=header)
    # Raise an exception if the response contains an HTTP error status code
    response.raise_for_status()

print(response.json())

{'runtimeName': 'llmops_test_api_runtime_2', 'runtimeDescription': 'This is a custom runtime made for LLMOPS', 'runtimeType': 'ComputeInstance', 'status': 'Unavailable', 'statusMessage': 'Runtime creation in progress', 'fromExistingEndpoint': False, 'fromExistingDeployment': False, 'instanceCount': 1, 'computeInstanceName': 'llmops-compute', 'publishedPort': 0, 'targetPort': 8080, 'fromExistingCustomApp': True, 'customAppName': 'llmops-app-2', 'createdOn': '2024-01-22T19:29:54.5103776+00:00', 'modifiedOn': '2024-01-22T19:29:54.5103791+00:00', 'owner': {'userObjectId': 'eb3dae97-27df-42aa-8a3a-6b1ee270201a', 'userTenantId': 'bee4890a-277b-4803-8ae5-9a71aa452cc8', 'userName': '7ef937d5-fea8-42c8-9291-98b1afc866c9'}}
